In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.utils import to_categorical
from keras import layers
from keras.optimizers import Adam , rmsprop

C:\Users\Ayaz\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
seed=7
np.random.seed(seed)

In [3]:
sonar_data=pd.read_csv('sonar.csv', header=None)

In [4]:
dataset=sonar_data.values

### Splitting Training  & Testing data 

In [5]:
X=dataset[:,0:60].astype('float')
Y=dataset[:,60]


### Converting Labels  

In [6]:
Le=LabelEncoder() 
Le.fit(Y) 
encoded=Le.transform(Y)


### Baseline Model

In [7]:
def create_baseline():
    model=Sequential()
    model.add(Dense(60, activation='relu',input_shape=(60,)))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer=Adam(lr=0.002), loss='binary_crossentropy', metrics=['accuracy'])
    return model

### Evaluating Baseline Model 

In [9]:
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Results: 83.73% (4.76%)


### Evaluate baseline model with standardized dataset

In [10]:
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=20, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Standardized: 85.59% (7.12%)


### Creating smaller model

In [1]:
def create_smaller():
    model=Sequential()
    model.add(Dense(30, activation='relu',input_shape=(60,)))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer=Adam(lr=0.002), loss='binary_crossentropy', metrics=['accuracy'])
    return model

### Evaluate Smaller model

In [12]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Smaller: 82.11% (7.42%)


### Creating Larger model

In [15]:
def create_larger():
    model=Sequential()
    model.add(Dense(60, activation='relu',input_shape=(60,)))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer=Adam(lr=0.002), loss='binary_crossentropy', metrics=['accuracy'])
    return model

### Evaluate Larger model

In [16]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Larger: 85.52% (7.55%)


### Creating model that overfits

In [25]:
def create_overfitting_model():
    model=Sequential()
    model.add(Dense(60, activation='relu',input_shape=(60,)))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer=Adam(lr=0.08), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [26]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_overfitting_model, epochs=200, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=20, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded, cv=kfold)
print(" %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

 53.23% (5.11%)


### Retuning the baseline model 

In [27]:
def create_retuned_model():
    model=Sequential()
    model.add(Dense(60, activation='relu',input_shape=(60,)))
    model.add(Dense(40, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_retuned_model, epochs=200, batch_size=10, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=20, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded, cv=kfold)
print("Result: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

### Model using keras functional API

In [23]:
import keras

def functional_api():
    inputs=keras.Input(shape=(60,))
    hl_1=layers.Dense(60,activation='relu')(inputs)
    hl_2=layers.Dense(30, activation='relu')(hl_1)
    outputs=layers.Dense(1, activation='sigmoid')(hl_2)
    model=keras.Model(inputs,outputs)
    model.compile(optimizer=Adam(lr=0.002),loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [79]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=functional_api, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded, cv=kfold)
print("Functional_API_Result: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Functional_API_Result: 85.56% (8.01%)


### Create model subclassing

In [9]:
import keras
class My_model(keras.Model):
    def __init__(self):
        super(My_model, self).__init__(keras.Input,self.output)
        self.input=layers.Dense(60, activation='relu')
        self.dense2=layers.Dense(30, activation='relu')
        self.output=layers.Dense(1, activation='sigmoid')
        
    def call(self):
        x1=self.dense1
        x2=self.dense2(x1)
        return self.dense3(x2)

In [12]:
class MyModel(keras.Model):

    def __init__(self):
        self.dense1 = layers.Dense(60, activation='relu')
        self.dense2 = layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)

model1 = MyModel()

In [13]:
#model1=My_model()
model1.compile(optimizer='adam' , loss='binary_crossentropy', metrics=['accuracy'])
model1.fit(X, encoded, epochs=20, batch_size=5)

AttributeError: 'MyModel' object has no attribute 'outputs'

### Applying kfold cross-validation without scikit-learn

In [8]:
k = 4
num_val_samples = len(X) // k
num_epochs = 100
all_scores = []

In [ ]:
for i in range(k):
    print('processing fold #', i)
    val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = encoded[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate([X[:i * num_val_samples],X[(i + 1) * num_val_samples:]],axis=0)
    partial_train_targets = np.concatenate([encoded[:i * num_val_samples],encoded[(i + 1) * num_val_samples:]],axis=0)
model = create_baseline()
model.fit(partial_train_data, partial_train_targets,epochs=num_epochs, batch_size=1, verbose=0)
val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
all_scores.append(val_mae)

In [14]:
np.mean(all_scores)*100

NameError: name 'all_scores' is not defined

In [15]:
np.std(all_scores)*100

NameError: name 'all_scores' is not defined